In [56]:
##Libraries##
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import datetime
import bccr
from bccr import PW
from bccr import SW
import lxml
from bs4 import BeautifulSoup

In [ ]:
##Connection to cmv_uat database##


server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [52]:
tradedate = '2021-07-12'  ###for risk Calcs###

monexbid = 22000.00
monexbidpp = 620.10

monexask = 20000.00
monexaskpp = 620.40

In [55]:
##Play to update risk and PnL##

##Blueprint   be carefull###
monexpp = float(1*SW(3439).iloc[-1])
#monexpp = 620.44 ## default to manual input
query = "SELECT * FROM [dbo].[fxblotter.t]"
# Read the SQL query into a DataFrame
fxblotter = pd.read_sql(query, connection, index_col='tradeid', parse_dates=True)
fxblotter.sort_index(inplace=True)
#datetime format#
fxblotter['tradedate'] = pd.to_datetime(fxblotter['tradedate'])
fxblotter['valuedate'] = pd.to_datetime(fxblotter['valuedate'])
## filtro para fecha en T o dia de consulta ### 
fxblottert = fxblotter.loc[fxblotter['tradedate'] == tradedate]
## filtro para accionid, escencialmente deja los Lost deals por fuera del calculo ##
fxblottert = fxblottert[fxblottert['accionid'] == 1] ### acepting 'addnew', 'lost' do not count for spread analysis##
###FX Posicion calculations###
bidtotalcliente = fxblottert.loc[fxblotter['comccyid'] == 1, 'montocom'].sum()
asktotalcliente = fxblottert.loc[fxblotter['venccyid'] == 1, 'montoven'].sum()
bidtotal = (fxblottert.loc[fxblottert['comccyid'] == 1, 'montocom'].sum() + monexbid)
asktotal = (fxblottert.loc[fxblottert['venccyid'] == 1, 'montoven'].sum() + monexask)
###fxnetposclientes = bidtotal - asktotal ### USD Moneda base ###
fxnetpos = (bidtotal) - (asktotal) ### USD Moneda base ###
fxnetposclientes = (bidtotalcliente) - (asktotalcliente) ### USD Moneda base ###
## Weighted average 
bidclientpp = fxblottert.loc[fxblottert['comccyid'] == 1, 'montoven'].sum() / bidtotalcliente
askclientpp = fxblottert.loc[fxblottert['venccyid'] == 1, 'montocom'].sum() / asktotalcliente
bidpp = (fxblottert.loc[fxblottert['comccyid'] == 1, 'montoven'].sum() +(monexbid*monexbidpp)) / bidtotal
askpp = (fxblottert.loc[fxblottert['venccyid'] == 1, 'montocom'].sum() +(monexask*monexaskpp)) / asktotal
##PnL##
if fxnetpos > 0:   
    coveredpnl =  asktotal * (askpp - bidpp)
    uncoveredpnl = (bidtotal - asktotal) * (monexpp - bidpp)  
else :
    coveredpnl =  bidtotal * (askpp - bidpp)
    uncoveredpnl = (asktotal - bidtotal) * (askpp - monexpp)
        ###uncoveredpnl

        ##if fxnetpos > 0 :   
        ##else :   
    
totalpnl = coveredpnl + uncoveredpnl

##print(f"daily total pnl is: {totalpnl}")
##print(f"FX net position:  {fxnetpos}")

print1 ="FX net position:  {:,} USD"
print(print1.format(round(fxnetpos,3)))
print("                                         ")
print2 ="FX clients net position:  {:,} USD"
print(print2.format(round(fxnetposclientes,3)))
print("                                         ")
print("************************************************************************************")
print("                                         ")
print3 ="Bid client amount:  {:,} @ {}"
print(print3.format(round(bidtotalcliente,3), round(bidclientpp,3)))
print("                                         ")
print4 ="Ask client amount:  {:,} @ {}"
print(print4.format(round(asktotalcliente,3), round(askclientpp,3)))
print("                                         ")
print("************************************************************************************") 
print5 ="Bid total amount:  {:,} @ {}"
print(print5.format(round(bidtotal,3), round(bidpp,3)))
print("                                         ")
print6 ="Ask total amount:  {:,} @ {}"
print(print6.format(round(asktotal,3), round(askpp,3)))
print("                                         ")
print("************************************************************************************")
print7 ="daily total pnl is:  {:,} crc"
print(print7.format(round(totalpnl,3)))

##fxblottert.head(100)

FX net position:  187.977 USD
                                         
FX clients net position:  -1,812.023 USD
                                         
************************************************************************************
                                         
Bid client amount:  52,168.51 @ 618.167
                                         
Ask client amount:  53,980.533 @ 623.574
                                         
************************************************************************************
Bid total amount:  74,168.51 @ 618.74
                                         
Ask total amount:  73,980.533 @ 622.716
                                         
************************************************************************************
daily total pnl is:  294,351.037 crc


In [51]:
##Deal Management Blotter###
qtrades = fxblottert['sideid'].value_counts()
qestado = fxblottert['estadoid'].value_counts()
print(qtrades)
print(qestado)


1    9
2    5
Name: sideid, dtype: int64
1    14
Name: estadoid, dtype: int64


## FX Posicion

<mark>Import fxblotter.t as a DF to make calculations</mark>

<mark>netpos = \[monto total de ventas (USD moneda base)\]  -  \[monto total de compra (USD moneda base)\]</mark>

## Stats

asktotal y askpp 

bidtotal y bidpp

## PnL 

PnL =  min \[total offer ,total bid \] \* \[ pp de offer  -  pp de bid\]  + if \[netpos \>0 , (bidtotal - offertotal) \* (monex.pp - bidpp), (offertotal - bidtotal) \* (offerpp - monexpp) \]

## DMB

compras, ventas, cantidad por 'estado', monex compra venta

## <span style="font-size: 14px;">definir compras y ventas de monex como variables float insert</span>

<span style="font-size: 14px;"><br></span>